In [18]:
import os

In [19]:
%pwd

'd:\\Repositories\\ML-geo'

In [20]:
os.chdir("d:/Repositories/ML-geo/")

In [37]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    transformation_path: Path

In [22]:
from XGboost_for_slopes.constants import *
from XGboost_for_slopes.utils.common import read_yaml, create_directories

In [38]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])


    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            transformation_path= config.transformation_path
            )
        return data_transformation_config

In [39]:
import os 
from XGboost_for_slopes import logger
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV, train_test_split
import dill
from src.XGboost_for_slopes.utils.common import save_object

In [45]:
class DataTransformation():
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def get_data_transformer_object(self):

        """
        This function is responsible for data transformation
        """
        try:

            num_Pipeline = Pipeline(
                steps=[
                ("imputer", SimpleImputer(strategy="median") ),
                ("scaler", StandardScaler()),
                ("polynomial-scaler", PolynomialFeatures(degree=2))
                ])
            logger.info("Data Transformation file created")

            return num_Pipeline
        except Exception as e:
            raise e

    def get_train_test_data(self):
        data = pd.read_csv(self.config.data_path)
        data = data[data["FSs"] < 3] # Engineers recommendation
        train, test = train_test_split(data,test_size=0.3)
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index= False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index= False)

        logger.info("Splitted data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)
        
    def initiate_data_transformation(self):
        try:

            logger.info("Obtaining preprocessing object")

            preprocessing_obj = self.get_data_transformer_object()
            save_object(self.config.transformation_path, preprocessing_obj)
            
        except:
            pass

In [33]:
def save_object(file_path, obj):
    try:
        dir_path= os.path.dirname(file_path)
        os.makedirs(dir_path, exist_ok=True)

        with open(file_path, "wb") as file_obj:
            dill.dump(obj, file_obj)

    except Exception as e:
        raise e
    


In [44]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config = data_transformation_config)
    data_transformation.get_train_test_data()
    data_transformation.initiate_data_transformation()

except Exception as e:
    raise e

[2023-12-07 01:24:12,822: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-07 01:24:12,824: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-07 01:24:12,826: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-07 01:24:12,827: INFO: common: created directory at: artifacts]
[2023-12-07 01:24:12,828: INFO: common: created directory at: artifacts/data_transformation]
[2023-12-07 01:24:13,209: INFO: 674148994: Splitted data into training and test sets]
[2023-12-07 01:24:13,210: INFO: 674148994: (38500, 9)]
[2023-12-07 01:24:13,211: INFO: 674148994: (16500, 9)]
[2023-12-07 01:24:13,212: INFO: 674148994: Obtaining preprocessing object]
[2023-12-07 01:24:13,212: INFO: 674148994: Columns standard scaling completed]
